In [95]:
#TODO
# detailed overview of other two data sets
# provide documentation of findings
# begin vizualizing
# Use tableau to add vizualized charts

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import read_csv # Importing the read_csv method 
                            # as it will allow us to write cleaner code
        
# Assigning variable to the datasets
# These names will represent the three different 
# data sets used to create this project
single_family_data = read_csv('./datasets/single_family_home.csv')
condo_data = read_csv('./datasets/median_condo_price.csv')
black_and_latino_data = read_csv('./datasets/black_and_latino_mortgage_rates.csv')
eviction_data = read_csv('./datasets/evictions.csv')
single_family_top_performers = read_csv('./datasets/single_fam_top_performers.csv')

# Format floats to show 2 decimal places for '% Change' col
pd.options.display.float_format = '{:.2f}'.format

# Prints a message to show that everything is loaded up
print("All systems GO!")

All systems GO!


In [96]:
# convert inputs in the '% change' col to float type
# Removing of the % symbol needs to happen first
single_family_data['% Change'] = single_family_data['% Change'].str[:-1]
condo_data['Condo.PercChange'] = condo_data['Condo.PercChange'].str[:-1]
black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'] = black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'].str[:-1]

# Converts the type to a float
single_family_data['% Change'] = single_family_data['% Change'].astype(float)
condo_data['Condo.PercChange'] = condo_data['Condo.PercChange'].astype(float)
black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'] = black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'].astype(float)


# Format floats to show 2 decimal places for '% Change' col
pd.options.display.float_format = '{:.2f}'.format

# Deletes notes column as its empty aside for one row
del single_family_data['Notes']

## Single Family Median Price Data:

In this section we will be combing through the single family median price dataset for the months of Jan-Jun of 2021 and the same months of 2022.

We have no aim as to what we hope to uncover, only merely to present the data in such a way that it makes full sense to the end user. 

In [97]:
# Getting a brief overview of the dataset
single_family_data.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,147.00,147.00,147.00
mean,721655.58,805912.24,10.79
std,380077.98,474227.22,9.58
min,347500.00,370250.00,-21.20
25%,491250.00,547000.00,5.65
50%,615000.00,660000.00,11.20
75%,801250.00,865000.00,15.60
max,3462500.00,4400000.00,44.40


In [98]:
# Top 5 performing municipalities overall
single_family_data.sort_values(by='% Change', ascending=False).head()

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
8,Somerville,Metro Core Communities,831125,1200000,44.40
13,Brookline,Streetcar Suburbs,1850000,2542000,37.40
141,Sherborn,Developing Suburbs,955000,1275000,33.50
125,Lincoln,Maturing Suburbs,1375000,1794000,30.50
144,Nahant,Maturing Suburbs,805000,1030000,28.00


In [99]:
# Worst 5 municipalities overall
single_family_data.sort_values(by='% Change', ascending=False).tail()

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
140,West Newbury,Developing Suburbs,810000,740000,-8.60
136,Marion,Developing Suburbs,607500,550000,-9.50
92,Ipswich,Developing Suburbs,747500,657500,-12.00
126,Groveland,Maturing Suburbs,615000,521500,-15.20
124,Rockport,Developing Suburbs,837500,660000,-21.20


In [100]:
# Presenting data by community type
metro_data_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Metro Core Communities"]
reg_urban_centers_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Regional Urban Centers"]
street_car_suburb_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Streetcar Suburbs"]
developing_suburb_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Developing Suburbs"]
maturing_suburb_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "'Maturing Suburbs"]
rural_town_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Rural Towns"]

In [101]:
# Sorting by community type column
# to present which type experienced
# the greatest increase in value by % 
single_metro_sorted_by_pct_change =  metro_data_single.sort_values(by='% Change', ascending=False)
single_reg_urban_pct_change = reg_urban_centers_single.sort_values(by='% Change', ascending=False)
single_street_car_suburn_pct_change = street_car_suburb_single.sort_values(by='% Change', ascending=False)
single_dev_suburb_pct_change = developing_suburb_single.sort_values(by='% Change', ascending=False)
maturing_suburb_single.sort_values(by='% Change', ascending=False)
rural_town_single.sort_values(by='% Change', ascending=False)

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
145,Ashby,Rural Towns,350000,370250,5.80


### Overview of Single Family home data by community type

In [102]:
# Metro Core Community
metro_data_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,7.00,7.00,7.00
mean,1135660.71,1399028.57,20.76
std,1091096.60,1397197.62,12.93
min,476500.00,577450.00,8.60
25%,531000.00,605375.00,11.25
50%,580000.00,630000.00,15.40
75%,1184312.50,1487500.00,27.20
max,3462500.00,4400000.00,44.40


In [103]:
metro_data_single.sort_values(by='% Change', ascending=False)

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
8,Somerville,Metro Core Communities,831125,1200000,44.40
27,Chelsea,Metro Core Communities,476500,606500,27.30
0,Boston,Metro Core Communities,3462500,4400000,27.10
1,Cambridge,Metro Core Communities,1537500,1775000,15.40
20,Everett,Metro Core Communities,537000,604250,12.50
16,Revere,Metro Core Communities,525000,577450,10.00
11,Malden,Metro Core Communities,580000,630000,8.60


In [104]:
# Regional Urban Centers Single Family
reg_urban_centers_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,17.00,17.00,17.00
mean,520102.94,567195.24,9.46
std,102287.47,103337.72,4.59
min,360000.00,413000.00,-0.80
25%,445000.00,500000.00,5.80
50%,529000.00,590000.00,10.30
75%,600000.00,620000.00,13.30
max,750000.00,815000.00,15.70


In [105]:
reg_urban_centers_single.sort_values(by='% Change', ascending=False)

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
78,Amesbury,Regional Urban Centers,481000,556319,15.70
6,Lawrence,Regional Urban Centers,360000,413000,14.70
3,Brockton,Regional Urban Centers,377500,430000,13.90
22,Salem,Regional Urban Centers,529000,600000,13.40
5,Lynn,Regional Urban Centers,450000,510000,13.30
10,Haverhill,Regional Urban Centers,435000,490000,12.60
19,Methuen,Regional Urban Centers,445000,500000,12.40
38,Norwood,Regional Urban Centers,600000,670000,11.70
26,Woburn,Regional Urban Centers,600000,662000,10.30
75,Newburyport,Regional Urban Centers,750000,815000,8.70


In [106]:
street_car_suburb_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,9.00,9.00,9.00
mean,1039227.78,1198755.56,12.99
std,439110.74,611463.45,10.78
min,629000.00,694900.00,1.60
25%,728050.00,802500.00,7.40
50%,790000.00,850000.00,9.70
75%,1441500.00,1575900.00,14.30
max,1850000.00,2542000.00,37.40


In [107]:
street_car_suburb_single.sort_values(by='% Change', ascending=False)

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
13,Brookline,Streetcar Suburbs,1850000,2542000,37.40
21,Arlington,Streetcar Suburbs,949500,1156000,21.70
15,Medford,Streetcar Suburbs,728050,832500,14.30
72,Winthrop,Streetcar Suburbs,629000,694900,10.50
43,Melrose,Streetcar Suburbs,775000,850000,9.70
50,Belmont,Streetcar Suburbs,1441500,1575900,9.30
7,Newton,Streetcar Suburbs,1490000,1600000,7.40
12,Waltham,Streetcar Suburbs,700000,735000,5.00
32,Watertown,Streetcar Suburbs,790000,802500,1.60


In [108]:
developing_suburb_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,65.00,65.00,65.00
mean,636039.23,687538.46,8.87
std,225127.99,242892.58,10.23
min,347500.00,400000.00,-21.20
25%,461000.00,517500.00,3.60
50%,583450.00,622500.00,9.50
75%,747500.00,764500.00,14.50
max,1425000.00,1631500.00,33.50


In [109]:
developing_suburb_single.sort_values(by='% Change', ascending=False)

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
141,Sherborn,Developing Suburbs,955000,1275000,33.50
79,Abington,Developing Suburbs,433000,550000,27.00
93,Medway,Developing Suburbs,532500,665000,24.90
116,Ayer,Developing Suburbs,418000,521500,24.80
123,Stow,Developing Suburbs,669000,833000,24.50
...,...,...,...,...,...
135,Manchester,Developing Suburbs,1162500,1100000,-5.40
140,West Newbury,Developing Suburbs,810000,740000,-8.60
136,Marion,Developing Suburbs,607500,550000,-9.50
92,Ipswich,Developing Suburbs,747500,657500,-12.00


In [110]:
rural_town_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,1.00,1.00,1.00
mean,350000.00,370250.00,5.80
std,NaN,NaN,NaN
min,350000.00,370250.00,5.80
25%,350000.00,370250.00,5.80
50%,350000.00,370250.00,5.80
75%,350000.00,370250.00,5.80
max,350000.00,370250.00,5.80


In [111]:
del single_family_top_performers['Notes']
single_family_top_performers['% Change'] = single_family_top_performers['% Change'].str[:-1]
single_family_data['% Change'] = single_family_data['% Change'].astype(float)
single_family_top_performers


,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
0,Somerville,Metro Core Communities,831125,1200000,44.4
1,Brookline,Streetcar Suburbs,1850000,2542000,37.4
2,Amesbury,Regional Urban Centers,481000,556319,15.7
3,Sherborn,Developing Suburbs,955000,1275000,33.5
4,Ashby,Rural Towns,350000,370250,5.8


## Condo Sales Data


In [112]:
# Getting a brief overview of the dataset
condo_data.describe()

#As seen below, running the above command
# We can see that we are working with the same qty of data inputs
# as for the single family homes with different col names
# we have already formatted the 'Condo.PercChange' column to allow for dtype
# changing which will allow us to more easily and correctly filter the data
# #

,Condo.2021,Condo.2022,Condo.PercChange
count,147.00,147.00,147.00
mean,428549.30,498465.06,10.94
std,209519.10,246305.55,18.64
min,0.00,0.00,-31.80
25%,313825.00,375750.00,-3.00
50%,415000.00,460000.00,8.30
75%,546250.00,642500.00,17.55
max,1000000.00,1290000.00,120.60


In [113]:
# condo_data.isna().sum()
# condo_data.isnull().sum()
# condo_data.columns


In [114]:
# Eviction rate is per 10,000
# so to calculate the % 
# I will need to divide rate by 100 and that will give us our percentage

# pd.unique(eviction_data['County'])
# eviction_data.sort_values(by='Jan-June 2022', ascending=False)

In [115]:
# Presenting data by community type
metro_data_condo = condo_data.loc[lambda condo_data: condo_data['Community Type'] == "Metro Core Communities"]
reg_urban_centers_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Regional Urban Centers"]
street_car_suburb_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Streetcar Suburbs"]
developing_suburb_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Developing Suburbs"]
maturing_suburb_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "'Maturing Suburbs"]
rural_town_single = single_family_data.loc[lambda single_family_data: single_family_data['Communities Type'] == "Rural Towns"]

In [116]:
metro_data_condo.describe()

,Condo.2021,Condo.2022,Condo.PercChange
count,7.00,7.00,7.00
mean,588778.57,634342.86,6.04
std,215419.48,265963.73,6.56
min,385500.00,407000.00,-3.20
25%,423500.00,423000.00,2.15
50%,440000.00,445000.00,5.60
75%,788475.00,871250.00,10.45
max,872000.00,999900.00,14.70
